# Processing the final dataset



In [107]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

In [108]:
# region of interest is Amazonia
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/raw/biomes_ibge").first().geometry()
# For specific biomes, the feature "id" is:
    # 18413 = Amazon
    # 18414 = Caatinga
    # 18415 = Cerrado
    # 18416 = Mata Atlantica
    # 18417 = Pampa
    # 18418 = Pantanal

In [110]:

land_use = ee.Image('projects/ee-ana-zonia/assets/land_use')
categorical = ee.Image("projects/ee-ana-zonia/assets/categorical")

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si").float()
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec").float()
cwd = ee.Image("projects/ee-ana-zonia/assets/raw/cwd_chave").float()
climate = yearly_si.addBands([mean_prec, cwd])

biomass = ee.Image('projects/ee-ana-zonia/assets/age_agbd')
total_export = age_agbd.addBands([land_use, climate, categorical]).updateMask(mask)

# export_image(climate, 'climate')

In [102]:
## stratify sample points per feature
stratified = final.stratifiedSample(
    numPoints = 10000,
    classBand = 'ecoreg',
    geometries = True
)

samples = image.sample(region=region)

# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toDrive(
    collection=samples,
    description='samplestst'
)
task.start()


In [111]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_123032_20140515').select(
    ['B4', 'B3', 'B2']
)

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

projection = landsat.select('B2').projection().getInfo()

# Export the image, specifying the CRS, transform, and region.
task = ee.batch.Export.image.toDrive(
    image=landsat,
    description='imageToDriveExample_transform',
    crs=projection['crs'],
    crsTransform=projection['transform'],
    region=geometry,
)
task.start()